In [1]:
import numpy as np
import pandas as pd

In [3]:
# data from: https://www.kaggle.com/yuanyuwendymu/airline-delay-and-cancellation-data-2009-2018?rvi=1

df16 = pd.read_csv('2016.csv')
df17 = pd.read_csv('2017.csv')
df18 = pd.read_csv('2018.csv')

# data from: https://www.kaggle.com/sobhanmoosavi/us-weather-events
df_weather = pd.read_csv('US_WeatherEvents_2016-2019.csv')

In [4]:
df = pd.concat([df16, df17, df18])

In [5]:
df.shape

(18505725, 28)

## 1. preprocessing

In [5]:
df_weather.head(10)

,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode
0,W-1,Snow,Light,2016-01-06 23:14:00,2016-01-07 00:34:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
1,W-2,Snow,Light,2016-01-07 04:14:00,2016-01-07 04:54:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
2,W-3,Snow,Light,2016-01-07 05:54:00,2016-01-07 15:34:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
3,W-4,Snow,Light,2016-01-08 05:34:00,2016-01-08 05:54:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
4,W-5,Snow,Light,2016-01-08 13:54:00,2016-01-08 15:54:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
5,W-6,Snow,Light,2016-01-08 16:14:00,2016-01-08 17:34:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
6,W-7,Fog,Severe,2016-01-09 12:54:00,2016-01-09 15:34:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
7,W-8,Snow,Light,2016-01-09 15:34:00,2016-01-09 16:14:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
8,W-9,Fog,Severe,2016-01-09 16:14:00,2016-01-09 16:34:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
9,W-10,Snow,Light,2016-01-09 16:34:00,2016-01-09 16:54:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0


In [6]:
#converting date format
df["date"] = pd.to_datetime(df["FL_DATE"]).dt.date

In [7]:
df_weather["date"] = pd.to_datetime(df_weather["StartTime(UTC)"]).dt.date

In [8]:
df.columns

Index(['FL_DATE', 'OP_CARRIER', 'OP_CARRIER_FL_NUM', 'ORIGIN', 'DEST',
       'CRS_DEP_TIME', 'DEP_TIME', 'DEP_DELAY', 'TAXI_OUT', 'WHEELS_OFF',
       'WHEELS_ON', 'TAXI_IN', 'CRS_ARR_TIME', 'ARR_TIME', 'ARR_DELAY',
       'CANCELLED', 'CANCELLATION_CODE', 'DIVERTED', 'CRS_ELAPSED_TIME',
       'ACTUAL_ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'CARRIER_DELAY',
       'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY',
       'Unnamed: 27', 'date'],
      dtype='object')

In [10]:
#converting airport code format in weather dataset
def convert_arpt_code(code):
    new_code = code[1:]
    return new_code

In [11]:
df_weather['Airport'] = df_weather['AirportCode'].apply(convert_arpt_code)

In [12]:
df_weather_dropped = df_weather.drop(columns = ['EventId','EndTime(UTC)','TimeZone','LocationLat','LocationLng','City','County','State','ZipCode','StartTime(UTC)','AirportCode'])

In [13]:
df_weather_dropped.head()

,Type,Severity,date,Airport
0,Snow,Light,2016-01-06,04V
1,Snow,Light,2016-01-07,04V
2,Snow,Light,2016-01-07,04V
3,Snow,Light,2016-01-08,04V
4,Snow,Light,2016-01-08,04V


In [14]:
#converting the format of the weather dataset
#so that it can be merged with the flights dataset
df_weather_dumb = pd.get_dummies(data = df_weather_dropped, columns=['Type', 'Severity'])

In [15]:
df_weather_dumb_1 = df_weather_dumb.groupby(["date", "Airport"], as_index = False).aggregate("sum")

In [16]:
df_combine_1 = df.merge(df_weather_dumb_1, left_on=['ORIGIN', 'date'], right_on=['Airport', 'date'], left_index = True, how = "left")

In [17]:
df_combine_1.columns

Index(['FL_DATE', 'OP_CARRIER', 'OP_CARRIER_FL_NUM', 'ORIGIN', 'DEST',
       'CRS_DEP_TIME', 'DEP_TIME', 'DEP_DELAY', 'TAXI_OUT', 'WHEELS_OFF',
       'WHEELS_ON', 'TAXI_IN', 'CRS_ARR_TIME', 'ARR_TIME', 'ARR_DELAY',
       'CANCELLED', 'CANCELLATION_CODE', 'DIVERTED', 'CRS_ELAPSED_TIME',
       'ACTUAL_ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'CARRIER_DELAY',
       'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY',
       'Unnamed: 27', 'date', 'Airport', 'Type_Cold', 'Type_Fog', 'Type_Hail',
       'Type_Precipitation', 'Type_Rain', 'Type_Snow', 'Type_Storm',
       'Severity_Heavy', 'Severity_Light', 'Severity_Moderate',
       'Severity_Other', 'Severity_Severe', 'Severity_UNK'],
      dtype='object')

In [18]:
#dropping unused columns
df_combine_drop = df_combine_1.drop(columns = ['FL_DATE', 
       'DEP_TIME', 'TAXI_OUT', 'WHEELS_OFF',
       'WHEELS_ON', 'TAXI_IN', 'ARR_TIME', 'ARR_DELAY',
       'CANCELLATION_CODE', 'DIVERTED', 
       'ACTUAL_ELAPSED_TIME', 'AIR_TIME', 'CARRIER_DELAY',
       'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY',
       'Unnamed: 27', 'Airport'])

In [19]:
#rename columns & join the two datasets again
#on Destination airport
df_combine_drop.rename(columns={'Type_Cold': 1, 'Type_Fog': 2, 'Type_Hail': 3,
       'Type_Precipitation': 4, 'Type_Rain': 5, 'Type_Snow': 6, 'Type_Storm': 7,
       'Severity_Heavy': 8, 'Severity_Light': 9, 'Severity_Moderate': 10,
       'Severity_Other': 11, 'Severity_Severe': 12, 'Severity_UNK': 13}, inplace = True)

In [20]:
df_combine_new = df_combine_drop.merge(df_weather_dumb_1, left_on=['DEST', 'date'], right_on=['Airport', 'date'], left_index = True, how = "left")

In [21]:
df_combine_new.rename(columns={'Type_Cold': 14, 'Type_Fog': 15, 'Type_Hail': 16,
       'Type_Precipitation': 17, 'Type_Rain': 18, 'Type_Snow': 19, 'Type_Storm': 20,
       'Severity_Heavy': 21, 'Severity_Light': 22, 'Severity_Moderate': 23,
       'Severity_Other': 24, 'Severity_Severe': 25, 'Severity_UNK': 26}, inplace = True)

In [22]:
df_combine_new.shape

(18505725, 38)

## 2a. feature engineering (cancellation model, no weather)
(After experimenting with the datasets, we found that appending the weather dataset does not improve model performance. Thus, the subsequent code only uses the original dataset containing flight information)

In [63]:
df_cancelled = df[df['CANCELLED'] == 1]

In [64]:
df_cancelled.shape

(265138, 29)

In [65]:
df_not_cancelled = df[df['CANCELLED'] == 0]
df_normal = df_not_cancelled.sample(265000)
df_cancel_model = pd.concat([df_cancelled, df_normal])

In [66]:
df_cancel_model.shape

(530138, 29)

In [67]:
df_cancel_model.drop(columns=['FL_DATE', 
       'DEP_TIME', 'TAXI_OUT', 'WHEELS_OFF',
       'WHEELS_ON', 'TAXI_IN', 'ARR_TIME', 'ARR_DELAY',
       'CANCELLATION_CODE', 'DIVERTED', 
       'ACTUAL_ELAPSED_TIME', 'AIR_TIME', 'CARRIER_DELAY',
       'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY',
       'Unnamed: 27', 'DEP_DELAY'], inplace=True)

In [69]:
df_cancel_model.head()

,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,CRS_ARR_TIME,CANCELLED,CRS_ELAPSED_TIME,DISTANCE,date
699,AS,29,ORD,SEA,1530,1815,1.0,285.0,1721.0,2016-01-01
710,AS,40,ADQ,ANC,1720,1809,1.0,49.0,253.0,2016-01-01
713,AS,49,ANC,ADQ,1530,1628,1.0,58.0,253.0,2016-01-01
1231,EV,3274,ERI,ORD,1655,1739,1.0,104.0,398.0,2016-01-01
1242,EV,3815,IAH,OKC,1445,1615,1.0,90.0,395.0,2016-01-01


In [70]:
df_cancel_model.isna().sum()

OP_CARRIER            0
OP_CARRIER_FL_NUM     0
ORIGIN                0
DEST                  0
CRS_DEP_TIME          0
CRS_ARR_TIME          0
CANCELLED             0
CRS_ELAPSED_TIME     16
DISTANCE              0
date                  0
dtype: int64

In [71]:
df_cancel_model = df_cancel_model.dropna(subset = ["CRS_ELAPSED_TIME"])

In [72]:
df_cancel_model['month'] = pd.to_datetime(df_cancel_model["date"]).dt.month
df_cancel_model['day'] = pd.to_datetime(df_cancel_model["date"]).dt.day
df_cancel_model['day_of_week'] = pd.to_datetime(df_cancel_model["date"]).dt.dayofweek

In [73]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [74]:
df_cancel_model['AIRLINE'] = le.fit_transform(df_cancel_model['OP_CARRIER'])
df_cancel_model['dest'] = le.fit_transform(df_cancel_model['DEST'])
df_cancel_model['origin'] = le.fit_transform(df_cancel_model['ORIGIN'])

In [76]:
df_cancel_model.drop(columns = ['date', 'DEST', 'ORIGIN', 'OP_CARRIER'], inplace = True)

## 3a. model training and prediction (cancellation model)

In [78]:
from sklearn.model_selection import train_test_split

In [79]:
y = df_cancel_model['CANCELLED']
X = df_cancel_model.drop(columns = ['CANCELLED'])
X_train, X_test, y_train, y_test = train_test_split(df_cancel_model, y, test_size=0.2, random_state=42)

In [80]:
X_train.shape

(424097, 12)

In [81]:
X_test.shape

(106025, 12)

In [82]:
from sklearn.linear_model import LogisticRegression

In [83]:
clf = LogisticRegression(random_state=0,solver='lbfgs',n_jobs=-1,verbose=100)

In [84]:
clf.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 96 concurrent workers.
Memmapping (shape=(424097, 12), dtype=float64) to new file /dev/shm/joblib_memmapping_folder_65789_9044805346/65789-140019227758776-9b2bb235df5347fa853e361bf4342078.pkl
Memmapping (shape=(424097,), dtype=float64) to new file /dev/shm/joblib_memmapping_folder_65789_9044805346/65789-140019227758776-dd4286b1afb9414bbbf19385b58609f0.pkl
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   36.8s
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   36.8s finished


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn', n_jobs=-1,
          penalty='l2', random_state=0, solver='lbfgs', tol=0.0001,
          verbose=100, warm_start=False)

In [85]:
predictions = clf.predict(X_test)
clf.score(X_test, y_test)

0.6012449893892949

In [86]:
from sklearn.ensemble import RandomForestClassifier

In [87]:
rf_clf = RandomForestClassifier(random_state=0,n_estimators=10,max_depth=10,n_jobs=-1)

In [88]:
rf_clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [89]:
predictions = rf_clf.predict(X_test)

In [ ]:
rf_clf.score(X_test, y_test)

In [ ]:
#using 10-fold cross validation
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, X, y, cv=10, verbose=10)

In [ ]:
scores.mean()

## 2b. feature engineering (delay model)

In [23]:
df_delay_model = df_combine_new.dropna(subset = ["DEP_DELAY", "CRS_ELAPSED_TIME"])

In [24]:
df_delay_model.fillna(0.0, inplace = True)

/usr/local/lib/python3.5/dist-packages/pandas/core/frame.py:4034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [25]:
df_delay_model.isna().sum()

OP_CARRIER           0
OP_CARRIER_FL_NUM    0
ORIGIN               0
DEST                 0
CRS_DEP_TIME         0
DEP_DELAY            0
CRS_ARR_TIME         0
CANCELLED            0
CRS_ELAPSED_TIME     0
DISTANCE             0
date                 0
1                    0
2                    0
3                    0
4                    0
5                    0
6                    0
7                    0
8                    0
9                    0
10                   0
11                   0
12                   0
13                   0
Airport              0
14                   0
15                   0
16                   0
17                   0
18                   0
19                   0
20                   0
21                   0
22                   0
23                   0
24                   0
25                   0
26                   0
dtype: int64

In [26]:
df_delay_model['month'] = pd.to_datetime(df_delay_model["date"]).dt.month
df_delay_model['day'] = pd.to_datetime(df_delay_model["date"]).dt.day
df_delay_model['day_of_week'] = pd.to_datetime(df_delay_model["date"]).dt.dayofweek

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

In [27]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [28]:
df_delay_model['AIRLINE'] = le.fit_transform(df_delay_model['OP_CARRIER'])
df_delay_model['dest'] = le.fit_transform(df_delay_model['DEST'])
df_delay_model['origin'] = le.fit_transform(df_delay_model['ORIGIN'])

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

In [29]:
df_delay_model.drop(columns = ['date', 'DEST', 'ORIGIN', 'OP_CARRIER', 'Airport'], inplace = True)

/usr/local/lib/python3.5/dist-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [30]:
df_delay_model.shape

(18244669, 39)

In [31]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
scaler = StandardScaler()

In [32]:
from sklearn.ensemble import RandomForestClassifier

In [33]:
df_delay_model.drop(columns=['CANCELLED'], inplace=True)

In [46]:
def f(row):
    if row['DEP_DELAY'] <= 0:
        val = 0
    elif row['DEP_DELAY'] <= 15 and row['DEP_DELAY'] > 0:
        val = 1
    elif row['DEP_DELAY'] <= 30 and row['DEP_DELAY'] > 15:
        val = 2
    else:
        val = 3
    return val

In [47]:
df_delay_model['DEPARTURE_DELAY_t'] = df_delay_model.apply(f, axis=1)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [49]:
df_delay_model[df_delay_model['DEPARTURE_DELAY_t'] == 3].shape

(2041629, 39)

In [150]:
df_0 = df_delay_model[df_delay_model['DEPARTURE_DELAY_t'] == 0]
de_not_0 = df_delay_model[df_delay_model['DEPARTURE_DELAY_t'] != 0]

In [152]:
df_0_new = df_0.sample(1100000)

In [156]:
df_new = pd.concat([df_0_new, de_not_0])

## 3b. model training and prediction (delay model)

In [50]:
y = df_delay_model['DEPARTURE_DELAY_t']
X = df_delay_model.drop(columns = ['DEP_DELAY', 'DEPARTURE_DELAY_t'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [51]:
X_train.shape

(14595735, 37)

In [130]:
clf = LogisticRegression(random_state=0,solver='lbfgs',n_jobs = -1,verbose = 100).fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 96 concurrent workers.
Memmapping (shape=(14595735, 37), dtype=float64) to new file /dev/shm/joblib_memmapping_folder_65789_5205661408/65789-140019227614568-ea02dce732ce43dbb89eb7a3bf821ddb.pkl
Memmapping (shape=(14595735,), dtype=int64) to new file /dev/shm/joblib_memmapping_folder_65789_5205661408/65789-140019227614568-6d20e8c9ad1b4f94a97aae1539bdf51b.pkl
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed: 21.0min
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed: 21.0min finished


In [260]:
clf.score(X_test, y_test)

0.6571624128195726

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state=0,n_estimators = 100,n_jobs = -1, verbose = 10)
clf.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 96 concurrent workers.


building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100building tree 38 of 100building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
bui

In [42]:
predictions = clf.predict(X_test)
clf.score(X_test, y_test)

[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done   8 out of 100 | elapsed:   14.9s remaining:  2.9min
[Parallel(n_jobs=96)]: Done  19 out of 100 | elapsed:   15.1s remaining:  1.1min
[Parallel(n_jobs=96)]: Done  30 out of 100 | elapsed:   15.4s remaining:   35.8s
[Parallel(n_jobs=96)]: Done  41 out of 100 | elapsed:   16.4s remaining:   23.5s
[Parallel(n_jobs=96)]: Done  52 out of 100 | elapsed:   18.9s remaining:   17.5s
[Parallel(n_jobs=96)]: Done  63 out of 100 | elapsed:   20.8s remaining:   12.2s
[Parallel(n_jobs=96)]: Done  74 out of 100 | elapsed:   22.1s remaining:    7.8s
[Parallel(n_jobs=96)]: Done  85 out of 100 | elapsed:   24.1s remaining:    4.3s
[Parallel(n_jobs=96)]: Done  96 out of 100 | elapsed:   25.1s remaining:    1.0s
[Parallel(n_jobs=96)]: Done 100 out of 100 | elapsed:   27.1s finished
[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done   8 out 

0.7064589274566215

In [ ]:
#using 10-fold cross validation
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, X, y, cv=5, verbose=10)